## ライブラリ導入

In [ ]:
from qiskit import *
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from commonLogic import *
from maxCut_common import *

%matplotlib inline

## IBM Quantum Platform設定

In [ ]:
# チャンネル (ibm_cloud/ibm_quantum)
channel = "ibm_quantum"

# バックエンド
backend_name = "ibm_osaka"

# APIトークン
token=""

## パラメータ設定

In [ ]:
# ノード数
num = 5
# エッジ構造
edges = [(0,1),(0,2),(1,2),(1,3),(2,4),(3,4)]
# 重み
weights = [1,1,1,1,1,1]

# (γ,β)最適値
args = np.array()

# グラフ描画

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
colors = ['aquamarine' for i in range(num)]
nx.draw(G, pos, with_labels=True, node_color=colors)
plt.show()

## 実装

In [ ]:
# IBMのサービスにアクセス
service = QiskitRuntimeService(channel=channel, token=token)
backend = service.backend(backend_name)

In [ ]:
# 回路作成
qc = get_mainCircuit(num,edges,weights,args)
qc.measure(range(num),range(num))
qc_transpiled = transpile(qc,backend)

In [ ]:
# 実機実行
sampler = Sampler(backend)
result = sampler.run([qc_transpiled]).result()
pub_result = result[0]

counts = pub_result.data.c.get_counts()
for i in range(2**num):
    binary_string = format(i, f'0{num}b')
    if binary_string not in counts:
        counts[binary_string]=0
counts = dict(sorted(counts.items()))

my_plot_histogram(counts,gradation=True)

## 分割後のグラフ

In [ ]:
max_key = max(counts, key=counts.get)

node_colors = []
for i in max_key:
    if i=='0': node_colors.append('aquamarine')
    else: node_colors.append('orange')

edge_colors = []
for u, v in G.edges():
    if node_colors[u] != node_colors[v]: edge_colors.append('red')
    else: edge_colors.append('black')

nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color=edge_colors)
plt.show()